# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [27]:
import graphlab as gl
import math
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to pauldeepakraj.r@gmail.com and will expire on August 06, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1473553312.log


In [30]:
sales_data = sales
gl.canvas.set_target("ipynb")

In [45]:
sales_data["bedrooms_squared"] = sales_data["bedrooms"]*sales_data["bedrooms"]
sales_data["bed_bath_rooms"] = sales_data["bedrooms"]*sales_data["bathrooms"]
sales_data["log_sqft_living"] = np.log(sales_data["sqft_living"])
sales_data["lat_plus_long"] = sales_data["lat"]+sales_data["long"]
sales_data.show()

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [46]:
train_data,test_data = sales_data.random_split(.8,seed=0)

In [47]:
test_data.show()

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [37]:
first_features = ['sqft_living', 'bedrooms', 'bathrooms','lat','long']
second_features = ['sqft_living', 'bedrooms', 'bathrooms','lat','long','bed_bath_rooms']
third_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms', 
                  'bedrooms_squared','log_sqft_living','lat_plus_long']

first_model = graphlab.linear_regression.create(train_data, target = 'price', features = first_features, 
                                                  validation_set = None)
second_model = graphlab.linear_regression.create(train_data, target = 'price', features = second_features, 
                                                  validation_set = None)
third_model = graphlab.linear_regression.create(train_data, target = 'price', features = third_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.143451     | 3190659.608868     | 228020.710401 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [38]:
print first_model.get("coefficients")
print second_model.get("coefficients")
print third_model.get("coefficients")

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -56140675.7444 | 1649985.42028 |
| sqft_living |  None | 310.263325778  | 3.18882960408 |
|   bedrooms  |  None | -59577.1160683 | 2487.27977322 |
|  bathrooms  |  None | 13811.8405419  | 3593.54213297 |
|     lat     |  None | 629865.789485  | 13120.7100323 |
|     long    |  None | -214790.285186 | 13284.2851607 |
+-------------+-------+----------------+---------------+
[6 rows x 4 columns]

+----------------+-------+----------------+---------------+
|      name      | index |     value      |     stderr    |
+----------------+-------+----------------+---------------+
|  (intercept)   |  None | -54410676.1152 | 1650405.16541 |
|  sqft_living   |  None | 304.449298056  | 3.20217535637 |
|    bedrooms    |  None | -116366.04323  | 4805.54966546 |
|   bathrooms    |  None | -77972.3305131 | 7565

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [40]:
first_predictions = first_model.predict(train_data)
print first_predictions[0] # should be 271789.505878

245784.01808


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [41]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    model_predictions = model.predict(data)
    
    # Then compute the residuals/errors
    residuals = outcome - model_predictions
        
    # Then square and add them up
    squared_residuals = residuals * residuals
    
    RSS = squared_residuals.sum()
    
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [43]:
print get_residual_sum_of_squares(first_model, train_data, train_data['price'])
print get_residual_sum_of_squares(second_model, train_data, train_data['price'])
print get_residual_sum_of_squares(third_model, train_data, train_data['price'])


9.71328233544e+14
9.61592067856e+14
9.03854036958e+14


In [44]:
print get_residual_sum_of_squares(first_model, test_data, test_data['price'])
print get_residual_sum_of_squares(second_model, test_data, test_data['price'])
print get_residual_sum_of_squares(third_model, test_data, test_data['price'])


2.26568089093e+14
2.24368799994e+14
2.55723991554e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [ ]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [ ]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [ ]:
# Learn the three models: (don't forget to set validation_set = None)
# Please see above

In [ ]:
# Examine/extract each model's coefficients:
# Please see above

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [ ]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
# Please see above

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:
# Please see above

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected?Think about the features that were added to each model from the previous.